In [ ]:
features used for the first submission
    features = [ 'Helpfulness', 
                'HelpfulnessDenominator', 
                'Average_Title_Sentiment' , 
                'Average_Text_Sentiment' , 
                'Review_Length', 
                'Time', 
                'Vader_Sentiment_Title',
                'Vader_Sentiment_Text',
                'TextBlob_Sentiment_Title',
                'TextBlob_Sentiment_Text', 
                'Non_Alpha_Count']

In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    random_state=100
)

# Train the model
lgb_model.fit(X_train_select, Y_train)

# Predict and evaluate
Y_pred = lgb_model.predict(X_test_select)
accuracy = accuracy_score(Y_test, Y_pred) * 100
print(f"LightGBM Accuracy: {accuracy:.2f}%")
return lgb_model, Y_pred

In [ ]:
model = GradientBoostingClassifier(
                    n_estimators=100,
                    learning_rate=0.1,
                    max_depth=5
                )       
    # Train the model
    model.fit(X_train_select, Y_train)
                
        # Make predictions
    Y_test_predictions = model.predict(X_test_select)
    # Calculate and display the final accuracy
    final_accuracy = accuracy_score(Y_test, Y_test_predictions) * 100
    print(f"\nFinal Accuracy (Using Best Gradient Boosting Model): {final_accuracy:.2f}%")
    return model, Y_test_predictions

In [ ]:
param_grid = {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 4, 5]}
    # Initialize variables to track the best model and accuracy
    best_model = None
    best_accuracy = 0
    best_params = {}

    # Custom grid search loop
    for n_estimators in param_grid['n_estimators']:
        for learning_rate in param_grid['learning_rate']:
            for max_depth in param_grid['max_depth']:
                # Initialize the model with the current parameters
                model = GradientBoostingClassifier(
                    n_estimators=n_estimators,
                    learning_rate=learning_rate,
                    max_depth=max_depth
                )
                
                # Train the model
                model.fit(X_train_select, Y_train)
                
                # Make predictions
                Y_test_predictions = model.predict(X_test_select)
                
                # Calculate accuracy
                accuracy = accuracy_score(Y_test, Y_test_predictions) * 100
                print(f"Params: n_estimators={n_estimators}, "
                    f"learning_rate={learning_rate}, max_depth={max_depth} -> "
                    f"Accuracy: {accuracy:.2f}%")
                
                # Track the best model
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_model = model
                    best_params = {
                        'n_estimators': n_estimators,
                        'learning_rate': learning_rate,
                        'max_depth': max_depth
                    }

    print(f"\nBest Parameters: {best_params}")
    print(f"Best Accuracy: {best_accuracy:.2f}%")

In [ ]:
# Define classifiers and their parameter grids
classifiers = {
    'Logistic Regression': (LogisticRegression(max_iter=1000), {
        'C': [0.01, 0.1, 1, 10],
        'solver': ['liblinear', 'saga']
    }),
    'Random Forest': (RandomForestClassifier(), {
        'n_estimators': [50, 100],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    }),
    'Gradient Boosting': (GradientBoostingClassifier(), {
        'n_estimators': [50, 100],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 4, 5]
    }),
    'SVM': (SVC(), {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    })
}

results = {}

for name, (clf, param_grid) in classifiers.items():
    grid_search = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train_select, Y_train)
    results[name] = grid_search.best_score_

# Print the results
for model, score in results.items():
    print(f"{model}: Best CV Score: {score:.2f}")

In [ ]:
'''
# Function to find the optimal constant (e.g., via cross-validation)
def find_best_constant(X_train, Y_train, features):
    param_grid = {'constant': np.linspace(10, 1000, 10)}  # Constants between 10 and 1000
    best_score = -np.inf
    best_constant = None

    for c in param_grid['constant']:
        # Create a temporary copy of the training data with scaled sentiment
        X_train_temp = X_train.copy()
        X_train_temp['Sentiment_Score'] = X_train_temp['Sentiment_Score'] / c

        # Select only the relevant numerical features for the model
        X_train_features = X_train_temp[features]

        # Initialize the KNN classifier
        knn = KNeighborsClassifier(n_neighbors=12)

        # Evaluate the model using cross-validation (5 folds)
        scores = cross_val_score(knn, X_train_features, Y_train, cv=5, scoring='accuracy')
        mean_score = np.mean(scores)

        if mean_score > best_score:
            best_score = mean_score
            best_constant = c

    print(f"Best constant: {best_constant} with score: {best_score}")
    return best_constant
'''